## Merge and rename storm surge hazard files

In [ ]:
import os
import pathlib
import re
import shutil
import subprocess
from glob import glob

In [ ]:
# Change to incoming_data, storm surge data folder
# os.chdir("../incoming_data/Deltares Storm Surge")

In [ ]:
out_dir = "../../processed_data/hazards/deltares_storm_surge/"

For Dominica and St Lucia, rename only:

In [ ]:
for f in sorted(glob("**/*DMA*.tif", recursive=True)):
    fname = pathlib.Path(f).name
    rcp, epoch, rp = re.match(r".*RCP(\d\d)(\d+)_epsg_32620_RP_(\d+).*", fname).group(
        1, 2, 3
    )
    if epoch == "2010":
        rcp = "baseline"
    out_fname = f"coastal__epoch_{epoch}__rcp_{rcp}__rp_{rp}__isoa3_dma.tif"
    shutil.copy(f, os.path.join(out_dir, out_fname))

In [ ]:
for f in sorted(glob("**/*LCA*.tif", recursive=True)):
    fname = pathlib.Path(f).name
    rcp, epoch, rp = re.match(r".*RCP(\d\d)(\d+)_epsg_32620_RP_(\d+).*", fname).group(
        1, 2, 3
    )
    if epoch == "2010":
        rcp = "baseline"
    out_fname = f"coastal__epoch_{epoch}__rcp_{rcp}__rp_{rp}__isoa3_lca.tif"
    shutil.copy(f, os.path.join(out_dir, out_fname))

Merge multiple island grids. Maybe align on major island

GRD 629325:676575 1324055:1387180

VCT 666820:705220 1391170:1481725

In [ ]:
vct_groups = set()

for f in sorted(glob("**/*VCT*.tif", recursive=True)):
    if "wrapped" in f:
        continue
    fname = pathlib.Path(f).name

    part, rcp, epoch, rp = re.match(
        r".*VCT00(\d)RCP(\d\d)(\d+)_epsg_32620_RP_(\d+).tif", fname
    ).group(1, 2, 3, 4)
    vct_groups.add((rcp, epoch, rp))

    subprocess.run(
        ["rm", "-f", f.replace(".tif", ".wrapped.tif")],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        ["rm", "-f", fname.replace(".tif", ".wrapped.tif")],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        [
            "gdalwarp",
            "-te",
            "666820",
            "1391170",
            "705220",
            "1481725",
            "-co",
            "COMPRESS=LZW",
            f,
            fname.replace(".tif", ".wrapped.tif"),
        ],
        check=True,
    )
    print(fname)

In [ ]:
for rcp, epoch, rp in vct_groups:
    if epoch == "2010":
        out_rcp = "baseline"
    else:
        out_rcp = rcp

    out_fname = f"coastal__epoch_{epoch}__rcp_{out_rcp}__rp_{rp}__isoa3_vct.tif"

    subprocess.run(
        ["rm", "-f", out_fname],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        [
            "gdal_merge.py",
            "-o",
            out_fname,
            "-co",
            "COMPRESS=LZW",
            "-a_nodata",
            "0",
            f"SaintVincentandtheGrenadinesVCT001RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT002RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT003RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT004RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT005RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT006RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT007RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"SaintVincentandtheGrenadinesVCT008RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
        ],
        check=True,
    )
    print(out_fname)

In [ ]:
grd_groups = set()

for f in sorted(glob("**/*GRD*.tif", recursive=True)):
    if "wrapped" in f:
        continue
    fname = pathlib.Path(f).name

    part, rcp, epoch, rp = re.match(
        r".*GRD00(\d)RCP(\d\d)(\d+)_epsg_32620_RP_(\d+).tif", fname
    ).group(1, 2, 3, 4)
    grd_groups.add((rcp, epoch, rp))

    subprocess.run(
        ["rm", "-f", f.replace(".tif", ".wrapped.tif")],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        ["rm", "-f", fname.replace(".tif", ".wrapped.tif")],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        [
            "gdalwarp",
            "-te",
            "629325",
            "1324055",
            "676575",
            "1387180",
            "-ts",
            "1575",
            "2104",
            "-co",
            "COMPRESS=LZW",
            f,
            fname.replace(".tif", ".wrapped.tif"),
        ],
        check=True,
    )
    print(fname)

In [ ]:
for rcp, epoch, rp in grd_groups:
    if epoch == "2010":
        out_rcp = "baseline"
    else:
        out_rcp = rcp

    out_fname = f"coastal__epoch_{epoch}__rcp_{out_rcp}__rp_{rp}__isoa3_grd.tif"

    subprocess.run(
        ["rm", "-f", out_fname],
        check=True,
        capture_output=True,
    )
    subprocess.run(
        [
            "gdal_calc.py",
            "-A",
            f"GrenadaGRD001RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            "-B",
            f"GrenadaGRD002RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            "-C",
            f"GrenadaGRD003RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            "-D",
            f"GrenadaGRD004RCP{rcp}{epoch}_epsg_32620_RP_{rp}.wrapped.tif",
            f"--outfile={out_fname}",
            '--calc="numpy.max((A,B,C,D),axis=0)"',
            "--NoDataValue=0",
            "--co=COMPRESS=LZW",
        ],
        check=True,
    )
    print(out_fname)